# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the citibike dataset - check that you are able to read in about 40k rows

In [1]:
# COMPATIBLE IMPORTS FOR PYTHON 2 AND 3

try:
    import urllib2 as ulib
except ImportError:
    import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

In [3]:
query = '''SELECT * FROM fb55.citibike; '''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


**TASK 1**

- Sort data by start_station_id, tripduration
  - Only checking trips with duration <= 3 hours
- Only show the top/last 10 records (aka head and tail in SQL) 
- List all unique start_station_id values
- Aggregation functions:
  - Count the number of trips (aka wc -l in SQL) 
  - Find the average/min/max trip duration

In [4]:
# sort my start station id ascending, trip duration descending 
# only looking at trips that are less than or equal to 3 hours

query = '''SELECT * FROM fb55.citibike
WHERE EXTRACT(HOUR FROM starttime::time) = 3
ORDER BY start_station_id ASC, tripduration DESC
LIMIT 10;
'''

table = get_data(query)
table.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,MacDougal St & Prince St,NaN,12790,-74.002971,846,2015-02-04 03:26:00+00,2015-02-04 03:40:00+00,128,12731,40.727103,509,9 Ave & W 22 St,40.745497,-74.001971,19896,Subscriber,1966,1
1,NaN,Cleveland Pl & Spring St,NaN,39676,-73.997203,779,2015-02-07 03:59:00+00,2015-02-07 04:12:00+00,151,39655,40.721816,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,14854,Subscriber,1988,1
2,NaN,E 40 St & 5 Ave,NaN,7996,-73.981632,520,2015-02-03 03:23:00+00,2015-02-03 03:32:00+00,153,7929,40.752062,518,E 39 St & 2 Ave,40.747804,-73.973442,16692,Subscriber,1964,1
3,NaN,LaGuardia Pl & W 3 St,NaN,253,-73.998102,383,2015-02-01 03:21:00+00,2015-02-01 03:27:00+00,161,185,40.729170,223,W 13 St & 7 Ave,40.737815,-73.999947,14631,Subscriber,1981,2
4,NaN,LaGuardia Pl & W 3 St,NaN,39667,-73.998102,315,2015-02-07 03:31:00+00,2015-02-07 03:36:00+00,161,39645,40.729170,438,St Marks Pl & 1 Ave,40.727791,-73.985649,17982,Subscriber,1983,1
5,NaN,E 39 St & 3 Ave,NaN,39666,-73.976049,699,2015-02-07 03:28:00+00,2015-02-07 03:40:00+00,167,39644,40.748901,237,E 11 St & 2 Ave,40.730473,-73.986724,17680,Subscriber,1988,1
6,NaN,W 16 St & The High Line,NaN,6569,-74.006818,799,2015-02-02 03:32:00+00,2015-02-02 03:45:00+00,212,6503,40.743349,478,11 Ave & W 41 St,40.760301,-73.998842,20180,Subscriber,1988,1
7,NaN,St Marks Pl & 2 Ave,NaN,39662,-73.987140,236,2015-02-07 03:20:00+00,2015-02-07 03:24:00+00,236,39640,40.728419,403,E 2 St & 2 Ave,40.725029,-73.990697,17547,Subscriber,1993,1
8,NaN,Lafayette St & Jersey St,NaN,12788,-73.995653,610,2015-02-04 03:21:00+00,2015-02-04 03:31:00+00,250,12729,40.724561,487,E 20 St & FDR Drive,40.733143,-73.975739,17787,Subscriber,1968,1
9,NaN,Lafayette St & Jersey St,NaN,252,-73.995653,422,2015-02-01 03:19:00+00,2015-02-01 03:26:00+00,250,184,40.724561,150,E 2 St & Avenue C,40.720874,-73.980858,14837,Subscriber,1980,1


In [5]:
table = get_data(query)
table.tail(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,MacDougal St & Prince St,NaN,12790,-74.002971,846,2015-02-04 03:26:00+00,2015-02-04 03:40:00+00,128,12731,40.727103,509,9 Ave & W 22 St,40.745497,-74.001971,19896,Subscriber,1966,1
1,NaN,Cleveland Pl & Spring St,NaN,39676,-73.997203,779,2015-02-07 03:59:00+00,2015-02-07 04:12:00+00,151,39655,40.721816,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,14854,Subscriber,1988,1
2,NaN,E 40 St & 5 Ave,NaN,7996,-73.981632,520,2015-02-03 03:23:00+00,2015-02-03 03:32:00+00,153,7929,40.752062,518,E 39 St & 2 Ave,40.747804,-73.973442,16692,Subscriber,1964,1
3,NaN,LaGuardia Pl & W 3 St,NaN,253,-73.998102,383,2015-02-01 03:21:00+00,2015-02-01 03:27:00+00,161,185,40.729170,223,W 13 St & 7 Ave,40.737815,-73.999947,14631,Subscriber,1981,2
4,NaN,LaGuardia Pl & W 3 St,NaN,39667,-73.998102,315,2015-02-07 03:31:00+00,2015-02-07 03:36:00+00,161,39645,40.729170,438,St Marks Pl & 1 Ave,40.727791,-73.985649,17982,Subscriber,1983,1
5,NaN,E 39 St & 3 Ave,NaN,39666,-73.976049,699,2015-02-07 03:28:00+00,2015-02-07 03:40:00+00,167,39644,40.748901,237,E 11 St & 2 Ave,40.730473,-73.986724,17680,Subscriber,1988,1
6,NaN,W 16 St & The High Line,NaN,6569,-74.006818,799,2015-02-02 03:32:00+00,2015-02-02 03:45:00+00,212,6503,40.743349,478,11 Ave & W 41 St,40.760301,-73.998842,20180,Subscriber,1988,1
7,NaN,St Marks Pl & 2 Ave,NaN,39662,-73.987140,236,2015-02-07 03:20:00+00,2015-02-07 03:24:00+00,236,39640,40.728419,403,E 2 St & 2 Ave,40.725029,-73.990697,17547,Subscriber,1993,1
8,NaN,Lafayette St & Jersey St,NaN,12788,-73.995653,610,2015-02-04 03:21:00+00,2015-02-04 03:31:00+00,250,12729,40.724561,487,E 20 St & FDR Drive,40.733143,-73.975739,17787,Subscriber,1968,1
9,NaN,Lafayette St & Jersey St,NaN,252,-73.995653,422,2015-02-01 03:19:00+00,2015-02-01 03:26:00+00,250,184,40.724561,150,E 2 St & Avenue C,40.720874,-73.980858,14837,Subscriber,1980,1


In [8]:
# list all unique start_station_id values special function called distinct 
query = '''
SELECT DISTINCT start_station_id
FROM fb55.citibike
'''

table = get_data(query)
table.head()

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN


In [15]:
#Aggregation functions:
#Count the number of trips (aka wc -l in SQL) 

query = '''SELECT COUNT(*)
FROM fb55.citibike; ''' 


table = get_data(query)
table.head(10)


,count,Unnamed: 1
0,46200,NaN


In [16]:
#Find the average/min/max trip duration

query = '''SELECT MAX(tripduration), MIN(tripduration), AVG(tripduration) from fb55.citibike'''

table = get_data(query)
table.head(10)

,max,min,avg
0,43016,60,675.865823


**TASK 2**

- Selecting trips started on Feb-02-2015 only 
- Selecting trips started on the weekends
- What are average trip duration during weekends? 
- Can we do the same for weekday?

In [44]:
# selecting trips that started on Feb 2 2015

query = '''
SELECT * FROM fb55.citibike
WHERE starttime::date = '2015-02-02';
'''

table = get_data(query)
table.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2


In [45]:
# selecting trips that began on the weekends

query = '''
SELECT * FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime::date) IN (0,6);
'''
table = get_data(query)
table.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0
5,NaN,E 20 St & 2 Ave,NaN,39894,-73.982050,296,2015-02-07 08:13:00+00,2015-02-07 08:18:00+00,461,39876,40.735877,435,W 21 St & 6 Ave,40.741740,-73.994156,17278,Subscriber,1987.0,1
6,NaN,Broadway & W 60 St,NaN,41512,-73.981918,600,2015-02-07 12:35:00+00,2015-02-07 12:45:00+00,499,41501,40.769155,523,W 38 St & 8 Ave,40.754666,-73.991382,15224,Subscriber,1959.0,1
7,NaN,E 6 St & Avenue D,NaN,41595,-73.976687,1203,2015-02-07 12:47:00+00,2015-02-07 13:07:00+00,411,41585,40.722281,428,E 3 St & 1 Ave,40.724677,-73.987834,21461,Subscriber,1968.0,1
8,NaN,Great Jones St,NaN,41806,-73.993790,520,2015-02-07 13:16:00+00,2015-02-07 13:25:00+00,229,41796,40.727434,168,W 18 St & 6 Ave,40.739713,-73.994564,14621,Subscriber,1964.0,1
9,NaN,W 13 St & 6 Ave,NaN,42316,-73.997044,487,2015-02-07 14:18:00+00,2015-02-07 14:26:00+00,345,42307,40.736494,509,9 Ave & W 22 St,40.745497,-74.001971,21362,Subscriber,1962.0,1


In [46]:
# finding average trip length on the weekends 

query = '''SELECT AVG(tripduration)
FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime::date) IN (0,6);
'''
table = get_data(query)
table.head(10)


,avg,Unnamed: 1
0,662.942181,NaN


In [47]:
# finding average trip on weekday


query = '''SELECT AVG(tripduration)
FROM fb55.citibike
WHERE EXTRACT(DOW FROM starttime::date) IN (1,2,3,4,5);
'''
table = get_data(query)
table.head(10)



,avg,Unnamed: 1
0,681.052292,NaN


**TASK 3**

- Showing the list of start station locations 
  - Using GROUP BY
- Show the number of trips started per station
  - but only for stations within 500m of Time Square!

The coordinates of Time Square are (40.7577,-73.9857)

In [52]:
# show list of start station locations 

query = '''SELECT start_station_name FROM fb55.citibike
GROUP BY start_station_name
'''

table = get_data(query)
table

,start_station_name,Unnamed: 1
0,E 30 St & Park Ave S,NaN
1,Pearl St & Anchorage Pl,NaN
2,W 56 St & 6 Ave,NaN
3,West Thames St,NaN
4,Bank St & Washington St,NaN
5,8 Ave & W 52 St,NaN
6,E 11 St & Broadway,NaN
7,DeKalb Ave & Skillman St,NaN
8,Atlantic Ave & Fort Greene Pl,NaN
9,Avenue D & E 8 St,NaN


In [62]:
# Show the number of trips started per station but only for stations within 500m of Time Square!
# Timesquare is 40.7577,-73.9857

query = '''SELECT start_station_name, COUNT(start_station_name) FROM fb55.citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
CDB_LatLng(40.7577, -73.9857)::geography, 500)
GROUP BY start_station_name;

'''

table = get_data(query)
table

,start_station_name,count
0,Broadway & W 41 St,251
1,Broadway & W 49 St,213
2,W 41 St & 8 Ave,507
3,W 42 St & 8 Ave,221
4,W 43 St & 6 Ave,112
5,W 45 St & 6 Ave,141
6,W 45 St & 8 Ave,141


**TASK 4**

Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!

In [79]:
# finding average trip length on the weekends 

query = '''SELECT AVG(tripduration)
FROM fb55.citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
CDB_LatLng(40.7577, -73.9857)::geography, 500)
AND EXTRACT(DOW FROM starttime::date) IN (0,6)
GROUP BY start_station_name
ORDER BY AVG(tripduration) DESC
LIMIT 1;
'''
table = get_data(query)
table.head(10)

,avg,Unnamed: 1
0,1010.104167,NaN


# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel